# week 3 part 3

 <h2> 11. Use geopy library to get the latitude and longitude values of Toronto</h2>

address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.

<h2>12. Create a map of Toronto with neighborhoods superimposed on top</h2>

In [ ]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

<h2> 13. Filter only boroughs that contain the word Toronto</h2>

In [ ]:
# filter borough names that contain the word Toronto
borough_names = list(toronto_df_new.Borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

In [ ]:
# create a new DataFrame with only boroughs that contain the word Toronto
toronto_df_new = toronto_df_new[toronto_df_new['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
print(toronto_df_new.shape)
toronto_df_new.head()

(38, 5)

    PostalCode   Borough          Neighborhood                     Latitude      Longitude
0   M4E          East Toronto     The Beaches                      43.676357     -79.293031
1   M4K          East Toronto     The Danforth West, Riverdale     43.679557     -79.352188
2   M4L          East Toronto     The Beaches West, India Bazaar   43.668999     -79.315572
3   M4M          East Toronto     Studio District                  43.659526     -79.340923
4   M4N          Central Toronto  Lawrence Park                    43.728020     -79.388790

In [ ]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

 <h2>14. Use the Foursquare API to explore the neighborhoods</h2>

In [ ]:
# define Foursquare Credentials and Version
CLIENT_ID = 'your Foursquare ID' # your Foursquare ID
CLIENT_SECRET = 'your Foursquare Secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: your Foursquare ID
CLIENT_SECRET:your Foursquare Secret

<h2>Now, let's get the top 100 venues that are within a radius of 500 meters.</h2>

In [ ]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['PostalCode'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [ ]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1695, 9)

In [ ]:
PostalCode	Borough	Neighborhood	BoroughLatitude	BoroughLongitude	VenueName	VenueLatitude	VenueLongitude	VenueCategory
0	M4E	East Toronto	The Beaches	43.676357	-79.293031	Grover Pub and Grub	43.679181	-79.297215	Pub
1	M4E	East Toronto	The Beaches	43.676357	-79.293031	Starbucks	43.678798	-79.298045	Coffee Shop
2	M4E	East Toronto	The Beaches	43.676357	-79.293031	Glen Stewart Ravine	43.676300	-79.294784	Other Great Outdoors
3	M4E	East Toronto	The Beaches	43.676357	-79.293031	Upper Beaches	43.680563	-79.292869	Neighborhood
4	M4K	East Toronto	The Danforth West, Riverdale	43.679557	-79.352188	Pantheon	43.677621	-79.351434	Greek Restaurant

 <h2>Let's check how many venues were returned for each PostalCode</h2>

In [ ]:
venues_df.groupby(["PostalCode", "Borough", "Neighborhood"]).count()

BoroughLatitude	BoroughLongitude	VenueName	VenueLatitude	VenueLongitude	VenueCategory
PostalCode	Borough	Neighborhood						
M4E	East Toronto	The Beaches	4	4	4	4	4	4
M4K	East Toronto	The Danforth West, Riverdale	42	42	42	42	42	42
M4L	East Toronto	The Beaches West, India Bazaar	21	21	21	21	21	21
M4M	East Toronto	Studio District	39	39	39	39	39	39
M4N	Central Toronto	Lawrence Park	4	4	4	4	4	4
M4P	Central Toronto	Davisville North	9	9	9	9	9	9
M4R	Central Toronto	North Toronto West	19	19	19	19	19	19
M4S	Central Toronto	Davisville	34	34	34	34	34	34
M4T	Central Toronto	Moore Park, Summerhill East	3	3	3	3	3	3
M4V	Central Toronto	Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West	14	14	14	14	14	14
M4W	Downtown Toronto	Rosedale	4	4	4	4	4	4
M4X	Downtown Toronto	Cabbagetown, St. James Town	45	45	45	45	45	45
M4Y	Downtown Toronto	Church and Wellesley	87	87	87	87	87	87
M5A	Downtown Toronto	Harbourfront, Regent Park	48	48	48	48	48	48
M5B	Downtown Toronto	Ryerson, Garden District	100	100	100	100	100	100
M5C	Downtown Toronto	St. James Town	100	100	100	100	100	100
M5E	Downtown Toronto	Berczy Park	55	55	55	55	55	55
M5G	Downtown Toronto	Central Bay Street	81	81	81	81	81	81
M5H	Downtown Toronto	Adelaide, King, Richmond	100	100	100	100	100	100
M5J	Downtown Toronto	Harbourfront East, Toronto Islands, Union Station	100	100	100	100	100	100
M5K	Downtown Toronto	Design Exchange, Toronto Dominion Centre	100	100	100	100	100	100
M5L	Downtown Toronto	Commerce Court, Victoria Hotel	100	100	100	100	100	100
M5N	Central Toronto	Roselawn	2	2	2	2	2	2
M5P	Central Toronto	Forest Hill North, Forest Hill West	4	4	4	4	4	4
M5R	Central Toronto	The Annex, North Midtown, Yorkville	23	23	23	23	23	23
M5S	Downtown Toronto	Harbord, University of Toronto	34	34	34	34	34	34
M5T	Downtown Toronto	Chinatown, Grange Park, Kensington Market	100	100	100	100	100	100
M5V	Downtown Toronto	CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara	13	13	13	13	13	13
M5W	Downtown Toronto	Stn A PO Boxes 25 The Esplanade	95	95	95	95	95	95
M5X	Downtown Toronto	First Canadian Place, Underground city	100	100	100	100	100	100
M6G	Downtown Toronto	Christie	15	15	15	15	15	15
M6H	West Toronto	Dovercourt Village, Dufferin	19	19	19	19	19	19
M6J	West Toronto	Little Portugal, Trinity	65	65	65	65	65	65
M6K	West Toronto	Brockton, Exhibition Place, Parkdale Village	19	19	19	19	19	19
M6P	West Toronto	High Park, The Junction South	23	23	23	23	23	23
M6R	West Toronto	Parkdale, Roncesvalles	16	16	16	16	16	16
M6S	West Toronto	Runnymede, Swansea	39	39	39	39	39	39
M7Y	East Toronto	Business Reply Mail Processing Centre 969 Eastern	19	19	19	19	19	19

 <h2>Let's find out how many unique categories can be curated from all the returned venues</h2>

In [ ]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 235 uniques categories.

In [ ]:
venues_df['VenueCategory'].unique()[:50]

In [ ]:
array(['Pub', 'Coffee Shop', 'Other Great Outdoors', 'Neighborhood',
       'Greek Restaurant', 'Ice Cream Shop', 'Cosmetics Shop',
       'Italian Restaurant', 'Yoga Studio', 'Health Food Store',
       'Brewery', 'Fruit & Vegetable Store', 'Pizza Place', 'Restaurant',
       'Juice Bar', 'Trail', 'Bookstore', 'Diner', 'Dessert Shop',
       'Bubble Tea Shop', 'Indian Restaurant', 'Spa', 'Grocery Store',
       'Bakery', 'Caribbean Restaurant', 'American Restaurant',
       'Liquor Store', 'Furniture / Home Store', 'Burger Joint', 'Gym',
       'Fish & Chips Shop', 'Park', 'Sushi Restaurant', 'Steakhouse',
       'Burrito Place', 'Pet Store', 'Fast Food Restaurant',
       'Movie Theater', 'Sandwich Place', 'Light Rail Station',
       'Food & Drink Shop', 'Fish Market', 'Cheese Shop', 'Café',
       'Comfort Food Restaurant', 'Chinese Restaurant',
       'Middle Eastern Restaurant', 'Stationery Store',
       'New American Restaurant', 'Seafood Restaurant'], dtype=object)

 <h2>15. Analyze Each Area</h2>

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['PostalCode'] = venues_df['PostalCode'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1695, 238)

In [ ]:
PostalCode	Borough	Neighborhoods	Adult Boutique	Afghan Restaurant	Airport	Airport Food Court	Airport Gate	Airport Lounge	Airport Service	Airport Terminal	American Restaurant	Antique Shop	Aquarium	Arepa Restaurant	Art Gallery	Art Museum	Arts & Crafts Store	Asian Restaurant	Auto Workshop	BBQ Joint	Baby Store	Bagel Shop	Bakery	Bank	Bar	Baseball Stadium	Basketball Stadium	Beach	Beer Bar	Beer Store	Belgian Restaurant	Bistro	Boat or Ferry	Bookstore	Boutique	Brazilian Restaurant	Breakfast Spot	Brewery	Bubble Tea Shop	Building	Burger Joint	Burrito Place	Bus Line	Butcher	Café	Cajun / Creole Restaurant	Caribbean Restaurant	Cheese Shop	Chinese Restaurant	Chocolate Shop	Church	Climbing Gym	Clothing Store	Cocktail Bar	Coffee Shop	College Arts Building	College Gym	College Rec Center	Comfort Food Restaurant	Comic Shop	Concert Hall	Convenience Store	Cosmetics Shop	Coworking Space	Creperie	Cuban Restaurant	Cupcake Shop	Dance Studio	Deli / Bodega	Department Store	Dessert Shop	Dim Sum Restaurant	Diner	Discount Store	Dog Run	Doner Restaurant	Donut Shop	Dumpling Restaurant	Eastern European Restaurant	Electronics Store	Ethiopian Restaurant	Event Space	Falafel Restaurant	Farmers Market	Fast Food Restaurant	Filipino Restaurant	Fish & Chips Shop	Fish Market	Flea Market	Food	Food & Drink Shop	Food Court	Food Truck	Fountain	French Restaurant	Fried Chicken Joint	Fruit & Vegetable Store	Furniture / Home Store	Gaming Cafe	Garden	Garden Center	Gastropub	Gay Bar	General Entertainment	General Travel	German Restaurant	Gift Shop	Gluten-free Restaurant	Gourmet Shop	Greek Restaurant	Grocery Store	Gym	Gym / Fitness Center	Harbor / Marina	Health & Beauty Service	Health Food Store	Historic Site	History Museum	Hobby Shop	Home Service	Hookah Bar	Hospital	Hostel	Hotel	Hotel Bar	Hotpot Restaurant	Ice Cream Shop	Indian Restaurant	Indie Movie Theater	Irish Pub	Italian Restaurant	Japanese Restaurant	Jazz Club	Jewelry Store	Jewish Restaurant	Juice Bar	Korean Restaurant	Lake	Latin American Restaurant	Light Rail Station	Lingerie Store	Liquor Store	Lounge	Mac & Cheese Joint	Malay Restaurant	Market	Martial Arts Dojo	Mediterranean Restaurant	Men's Store	Mexican Restaurant	Middle Eastern Restaurant	Miscellaneous Shop	Modern European Restaurant	Molecular Gastronomy Restaurant	Monument / Landmark	Movie Theater	Moving Target	Museum	Music Store	Music Venue	Neighborhood	New American Restaurant	Nightclub	Noodle House	Office	Opera House	Optical Shop	Organic Grocery	Other Great Outdoors	Outdoor Sculpture	Park	Performing Arts Venue	Persian Restaurant	Pet Store	Pharmacy	Piano Bar	Pizza Place	Playground	Plaza	Poke Place	Pool	Portuguese Restaurant	Poutine Place	Pub	Ramen Restaurant	Record Shop	Recording Studio	Rental Car Location	Restaurant	Sake Bar	Salad Place	Salon / Barbershop	Sandwich Place	Scenic Lookout	Sculpture Garden	Seafood Restaurant	Shoe Store	Shopping Mall	Skate Park	Skating Rink	Smoke Shop	Smoothie Shop	Snack Place	Soup Place	South American Restaurant	Southern / Soul Food Restaurant	Spa	Speakeasy	Sporting Goods Shop	Sports Bar	Stadium	Stationery Store	Steakhouse	Strip Club	Supermarket	Sushi Restaurant	Swim School	Taco Place	Tailor Shop	Taiwanese Restaurant	Tanning Salon	Tapas Restaurant	Tea Room	Tennis Court	Thai Restaurant	Theater	Thrift / Vintage Store	Toy / Game Store	Trail	Train Station	Vegetarian / Vegan Restaurant	Video Game Store	Vietnamese Restaurant	Wine Bar	Wings Joint	Women's Store	Yoga Studio
0	M4E	East Toronto	The Beaches	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0
1	M4E	East Toronto	The Beaches	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0
2	M4E	East Toronto	The Beaches	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0
3	M4E	East Toronto	The Beaches	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0
4	M4K	East Toronto	The Danforth West, Riverdale	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0

<h2>Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category</h2>

In [ ]:
toronto_grouped = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped

(38, 238)

In [ ]:
PostalCode	Borough	Neighborhoods	Adult Boutique	Afghan Restaurant	Airport	Airport Food Court	Airport Gate	Airport Lounge	Airport Service	Airport Terminal	American Restaurant	Antique Shop	Aquarium	Arepa Restaurant	Art Gallery	Art Museum	Arts & Crafts Store	Asian Restaurant	Auto Workshop	BBQ Joint	Baby Store	Bagel Shop	Bakery	Bank	Bar	Baseball Stadium	Basketball Stadium	Beach	Beer Bar	Beer Store	Belgian Restaurant	Bistro	Boat or Ferry	Bookstore	Boutique	Brazilian Restaurant	Breakfast Spot	Brewery	Bubble Tea Shop	Building	Burger Joint	Burrito Place	Bus Line	Butcher	Café	Cajun / Creole Restaurant	Caribbean Restaurant	Cheese Shop	Chinese Restaurant	Chocolate Shop	Church	Climbing Gym	Clothing Store	Cocktail Bar	Coffee Shop	College Arts Building	College Gym	College Rec Center	Comfort Food Restaurant	Comic Shop	Concert Hall	Convenience Store	Cosmetics Shop	Coworking Space	Creperie	Cuban Restaurant	Cupcake Shop	Dance Studio	Deli / Bodega	Department Store	Dessert Shop	Dim Sum Restaurant	Diner	Discount Store	Dog Run	Doner Restaurant	Donut Shop	Dumpling Restaurant	Eastern European Restaurant	Electronics Store	Ethiopian Restaurant	Event Space	Falafel Restaurant	Farmers Market	Fast Food Restaurant	Filipino Restaurant	Fish & Chips Shop	Fish Market	Flea Market	Food	Food & Drink Shop	Food Court	Food Truck	Fountain	French Restaurant	Fried Chicken Joint	Fruit & Vegetable Store	Furniture / Home Store	Gaming Cafe	Garden	Garden Center	Gastropub	Gay Bar	General Entertainment	General Travel	German Restaurant	Gift Shop	Gluten-free Restaurant	Gourmet Shop	Greek Restaurant	Grocery Store	Gym	Gym / Fitness Center	Harbor / Marina	Health & Beauty Service	Health Food Store	Historic Site	History Museum	Hobby Shop	Home Service	Hookah Bar	Hospital	Hostel	Hotel	Hotel Bar	Hotpot Restaurant	Ice Cream Shop	Indian Restaurant	Indie Movie Theater	Irish Pub	Italian Restaurant	Japanese Restaurant	Jazz Club	Jewelry Store	Jewish Restaurant	Juice Bar	Korean Restaurant	Lake	Latin American Restaurant	Light Rail Station	Lingerie Store	Liquor Store	Lounge	Mac & Cheese Joint	Malay Restaurant	Market	Martial Arts Dojo	Mediterranean Restaurant	Men's Store	Mexican Restaurant	Middle Eastern Restaurant	Miscellaneous Shop	Modern European Restaurant	Molecular Gastronomy Restaurant	Monument / Landmark	Movie Theater	Moving Target	Museum	Music Store	Music Venue	Neighborhood	New American Restaurant	Nightclub	Noodle House	Office	Opera House	Optical Shop	Organic Grocery	Other Great Outdoors	Outdoor Sculpture	Park	Performing Arts Venue	Persian Restaurant	Pet Store	Pharmacy	Piano Bar	Pizza Place	Playground	Plaza	Poke Place	Pool	Portuguese Restaurant	Poutine Place	Pub	Ramen Restaurant	Record Shop	Recording Studio	Rental Car Location	Restaurant	Sake Bar	Salad Place	Salon / Barbershop	Sandwich Place	Scenic Lookout	Sculpture Garden	Seafood Restaurant	Shoe Store	Shopping Mall	Skate Park	Skating Rink	Smoke Shop	Smoothie Shop	Snack Place	Soup Place	South American Restaurant	Southern / Soul Food Restaurant	Spa	Speakeasy	Sporting Goods Shop	Sports Bar	Stadium	Stationery Store	Steakhouse	Strip Club	Supermarket	Sushi Restaurant	Swim School	Taco Place	Tailor Shop	Taiwanese Restaurant	Tanning Salon	Tapas Restaurant	Tea Room	Tennis Court	Thai Restaurant	Theater	Thrift / Vintage Store	Toy / Game Store	Trail	Train Station	Vegetarian / Vegan Restaurant	Video Game Store	Vietnamese Restaurant	Wine Bar	Wings Joint	Women's Store	Yoga Studio
0	M4E	East Toronto	The Beaches	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.250000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.250000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.25	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.250000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.00000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000
1	M4K	East Toronto	The Danforth West, Riverdale	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.023810	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.023810	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.047619	0.000000	0.00	0.000000	0.023810	0.023810	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.023810	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.095238	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.023810	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.023810	0.00	0.023810	0.000000	0.000000	0.00	0.000000	0.00	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.02381	0.023810	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.238095	0.023810	0.000000	0.000000	0.000000	0.000000	0.023810	0.000000	0.000000	0.000000	0.0	0.00	0.00	0.000000	0.000000	0.00	0.00	0.071429	0.023810	0.000000	0.000000	0.047619	0.000000	0.000000	0.000000	0.000000	0.023810	0.000000	0.00	0.000000	0.000000	0.00	0.023810	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.023810	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.023810	0.000000	0.000000	0.000000	0.000000	0.023810	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.023810	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.02381	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.023810
2	M4L	East Toronto	The Beaches West, India Bazaar	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.047619	0.000000	0.00	0.047619	0.047619	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.047619	0.00	0.047619	0.000000	0.000000	0.000000	0.047619	0.00	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.047619	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.00	0.00	0.000000	0.000000	0.00	0.00	0.047619	0.000000	0.000000	0.000000	0.047619	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.047619	0.00	0.047619	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.047619	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.095238	0.000000	0.000000	0.047619	0.000000	0.000000	0.047619	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.047619	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.095238	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.047619	0.000000	0.000000	0.047619	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.00000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000
3	M4M	East Toronto	Studio District	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.051282	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.051282	0.025641	0.025641	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.025641	0.000000	0.00	0.000000	0.025641	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.102564	0.000000	0.000000	0.025641	0.025641	0.000000	0.000000	0.000000	0.025641	0.000000	0.076923	0.000000	0.000000	0.00	0.025641	0.000000	0.000000	0.025641	0.000000	0.025641	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.025641	0.000000	0.000000	0.00	0.000000	0.00	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.025641	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.00	0.000000	0.000000	0.025641	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.025641	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.00	0.00	0.000000	0.000000	0.00	0.00	0.025641	0.000000	0.000000	0.000000	0.051282	0.000000	0.000000	0.000000	0.000000	0.025641	0.000000	0.00	0.025641	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.025641	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.025641	0.000000	0.025641	0.025641	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.025641	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.025641	0.00	0.000000	0.025641	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.025641	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.00000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.025641
4	M4N	Central Toronto	Lawrence Park	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.25	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.25	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.250000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.25	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.00000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000
5	M4P	Central Toronto	Davisville North	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.111111	0.000000	0.000000	0.00	0.111111	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.111111	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.111111	0.00	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.111111	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.00	0.00	0.000000	0.222222	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.111111	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.111111	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.00000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000
6	M4R	Central Toronto	North Toronto West	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.052632	0.000000	0.000000	0.000000	0.052632	0.000000	0.105263	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.052632	0.00	0.052632	0.000000	0.000000	0.00	0.000000	0.00	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.052632	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.052632	0.000000	0.000000	0.000000	0.000000	0.0	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.052632	0.000000	0.052632	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.052632	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.052632	0.000000	0.000000	0.000000	0.052632	0.052632	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.052632	0.000000	0.105263	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.052632	0.00000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.052632
7	M4S	Central Toronto	Davisville	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.029412	0.000000	0.00	0.029412	0.000000	0.00	0.000000	0.058824	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.058824	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.088235	0.00	0.029412	0.029412	0.000000	0.00	0.000000	0.00	0.0000	0.000000	0.000000	0.000000	0.000000	0.029412	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.029412	0.00000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.029412	0.029412	0.000000	0.029412	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.029412	0.000000	0.000000	0.058824	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.029412	0.000000	0.000000	0.000000	0.058824	0.000000	0.058824	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.029412	0.000000	0.000000	0.000000	0.088235	0.00	0.000000	0.058824	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.058824	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.029412	0.000000	0.00	0.029412	0.00000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000
8	M4T	Central Toronto	Moore Park, Summerhill East	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.333333	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.333333	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.333333	0.000000	0.000000	0.00	0.000000	0.00000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000
9	M4V	Central Toronto	Deer Park, Forest Hill SE, Rathnelly, South Hi...	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.071429	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.071429	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.142857	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.071429	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.071429	0.00000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.071429	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.071429	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.142857	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.071429	0.000000	0.000000	0.000000	0.000000	0.071429	0.071429	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.00000	0.00	0.000000	0.000000	0.071429	0.000000	0.000000	0.00	0.000000
10	M4W	Downtown Toronto	Rosedale	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.500000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.250000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.25000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000
11	M4X	Downtown Toronto	Cabbagetown, St. James Town	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.044444	0.022222	0.000000	0.00	0.000000	0.000000	0.000000	0.022222	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.022222	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.022222	0.044444	0.000000	0.022222	0.000000	0.044444	0.000000	0.000000	0.000000	0.000000	0.000000	0.088889	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.022222	0.000000	0.000000	0.00	0.022222	0.000000	0.000000	0.00	0.000000	0.00	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.00	0.000000	0.000000	0.022222	0.000000	0.022222	0.00	0.00	0.022222	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.022222	0.000000	0.000000	0.044444	0.022222	0.000000	0.022222	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.022222	0.000000	0.000000	0.000000	0.022222	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.022222	0.022222	0.000000	0.000000	0.022222	0.022222	0.000000	0.044444	0.022222	0.022222	0.000000	0.000000	0.000000	0.000000	0.044444	0.000000	0.000000	0.000000	0.000000	0.088889	0.000000	0.000000	0.000000	0.022222	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.022222	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.022222	0.00	0.000000	0.000000	0.000000	0.022222	0.000000	0.00	0.000000	0.00000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000
12	M4Y	Downtown Toronto	Church and Wellesley	0.011494	0.011494	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.011494	0.000000	0.00	0.00	0.000000	0.000000	0.011494	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.011494	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.011494	0.000000	0.00	0.011494	0.000000	0.022989	0.00	0.034483	0.011494	0.00	0.000000	0.022989	0.000000	0.011494	0.000000	0.011494	0.000000	0.000000	0.000000	0.000000	0.000000	0.057471	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.011494	0.000000	0.000000	0.011494	0.000000	0.000000	0.000000	0.00	0.011494	0.000000	0.011494	0.00	0.000000	0.00	0.0000	0.000000	0.011494	0.000000	0.000000	0.000000	0.022989	0.00	0.000000	0.000000	0.000000	0.000000	0.011494	0.00	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.00	0.000000	0.000000	0.022989	0.045977	0.011494	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.011494	0.011494	0.000000	0.000000	0.011494	0.000000	0.000000	0.000000	0.011494	0.0	0.00	0.00	0.000000	0.011494	0.00	0.00	0.011494	0.011494	0.000000	0.000000	0.000000	0.068966	0.000000	0.000000	0.000000	0.011494	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.022989	0.022989	0.011494	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.011494	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.011494	0.000000	0.011494	0.011494	0.000000	0.011494	0.011494	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.022989	0.011494	0.000000	0.000000	0.000000	0.034483	0.011494	0.000000	0.011494	0.000000	0.00	0.011494	0.011494	0.000000	0.000000	0.000000	0.00	0.011494	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.011494	0.000000	0.000000	0.011494	0.011494	0.000000	0.057471	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.011494	0.000000	0.000000	0.011494	0.00	0.000000	0.00000	0.00	0.000000	0.011494	0.011494	0.000000	0.011494	0.00	0.011494
13	M5A	Downtown Toronto	Harbourfront, Regent Park	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.020833	0.00	0.00	0.020833	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.062500	0.020833	0.000000	0.00	0.000000	0.000000	0.000000	0.020833	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.041667	0.020833	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.062500	0.000000	0.000000	0.000000	0.000000	0.020833	0.000000	0.000000	0.000000	0.000000	0.166667	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.020833	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.0000	0.020833	0.000000	0.020833	0.000000	0.020833	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.020833	0.000000	0.00000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.020833	0.000000	0.000000	0.020833	0.000000	0.000000	0.020833	0.020833	0.000000	0.000000	0.0	0.00	0.00	0.000000	0.020833	0.00	0.00	0.020833	0.000000	0.000000	0.000000	0.020833	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.041667	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.062500	0.020833	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.062500	0.000000	0.000000	0.000000	0.000000	0.020833	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.020833	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.020833	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.041667	0.00	0.000000	0.00000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000
14	M5B	Downtown Toronto	Ryerson, Garden District	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.00	0.00	0.010000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.000000	0.010000	0.010000	0.020000	0.00	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.00	0.000000	0.000000	0.010000	0.00	0.010000	0.010000	0.00	0.000000	0.040000	0.000000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.080000	0.000000	0.090000	0.000000	0.000000	0.01	0.000000	0.010000	0.010000	0.000000	0.030000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.010000	0.00	0.020000	0.000000	0.000000	0.00	0.000000	0.00	0.0000	0.000000	0.000000	0.000000	0.010000	0.000000	0.010000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.01	0.000000	0.000000	0.000000	0.000000	0.00000	0.010000	0.00	0.000000	0.000000	0.010000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.010000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.01	0.00	0.000000	0.010000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.020000	0.020000	0.000000	0.000000	0.000000	0.010000	0.000000	0.01	0.000000	0.000000	0.01	0.000000	0.010000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.010000	0.030000	0.010000	0.010000	0.000000	0.00	0.010000	0.000000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.010000	0.00	0.000000	0.00	0.01	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.020000	0.000000	0.020000	0.000000	0.000000	0.000000	0.000000	0.010000	0.020000	0.000000	0.000000	0.000000	0.020000	0.000000	0.000000	0.000000	0.010000	0.00	0.000000	0.010000	0.010000	0.010000	0.000000	0.00	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.00	0.01	0.000000	0.000000	0.01	0.000000	0.020000	0.000000	0.020000	0.020000	0.00	0.010000	0.00000	0.00	0.010000	0.010000	0.010000	0.010000	0.000000	0.00	0.000000
15	M5C	Downtown Toronto	St. James Town	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.020000	0.000000	0.00	0.00	0.010000	0.000000	0.000000	0.000000	0.000000	0.020000	0.000000	0.010000	0.030000	0.010000	0.000000	0.00	0.000000	0.000000	0.020000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.00	0.030000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.050000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.040000	0.030000	0.070000	0.000000	0.000000	0.00	0.010000	0.000000	0.010000	0.000000	0.030000	0.000000	0.020000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.00	0.020000	0.000000	0.000000	0.00	0.000000	0.00	0.0000	0.010000	0.000000	0.000000	0.000000	0.020000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.010000	0.010000	0.000000	0.000000	0.00000	0.010000	0.00	0.000000	0.000000	0.030000	0.000000	0.000000	0.00	0.01	0.000000	0.00	0.000000	0.000000	0.010000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.00	0.00	0.010000	0.050000	0.00	0.00	0.010000	0.010000	0.000000	0.000000	0.030000	0.020000	0.010000	0.000000	0.000000	0.000000	0.010000	0.00	0.010000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.010000	0.000000	0.000000	0.010000	0.00	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.030000	0.010000	0.000000	0.000000	0.000000	0.000000	0.020000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.060000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.020000	0.000000	0.000000	0.000000	0.00	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.010000	0.000000	0.00	0.000000	0.000000	0.000000	0.010000	0.010000	0.00	0.000000	0.00000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.01	0.010000
16	M5E	Downtown Toronto	Berczy Park	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.018182	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.018182	0.036364	0.000000	0.000000	0.00	0.018182	0.018182	0.036364	0.000000	0.018182	0.018182	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.036364	0.000000	0.000000	0.036364	0.000000	0.000000	0.000000	0.000000	0.018182	0.054545	0.072727	0.000000	0.000000	0.00	0.018182	0.000000	0.018182	0.000000	0.018182	0.000000	0.018182	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.018182	0.000000	0.000000	0.00	0.000000	0.00	0.0000	0.000000	0.000000	0.000000	0.000000	0.036364	0.000000	0.00	0.000000	0.018182	0.000000	0.000000	0.000000	0.00	0.000000	0.018182	0.018182	0.000000	0.00000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.018182	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.00	0.00	0.000000	0.018182	0.00	0.00	0.000000	0.000000	0.000000	0.018182	0.036364	0.000000	0.018182	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.018182	0.018182	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.018182	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.018182	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.036364	0.000000	0.000000	0.000000	0.000000	0.054545	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.036364	0.000000	0.018182	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.036364	0.000000	0.000000	0.000000	0.00	0.00	0.018182	0.000000	0.00	0.000000	0.018182	0.000000	0.018182	0.000000	0.00	0.000000	0.00000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000
17	M5G	Downtown Toronto	Central Bay Street	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.012346	0.000000	0.00	0.00	0.000000	0.012346	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.012346	0.000000	0.037037	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.024691	0.00	0.037037	0.000000	0.00	0.000000	0.074074	0.000000	0.000000	0.000000	0.024691	0.000000	0.000000	0.000000	0.012346	0.000000	0.160494	0.000000	0.000000	0.00	0.000000	0.012346	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.012346	0.012346	0.00	0.012346	0.012346	0.000000	0.00	0.012346	0.00	0.0000	0.000000	0.000000	0.000000	0.024691	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.012346	0.000000	0.00000	0.012346	0.00	0.000000	0.000000	0.012346	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.012346	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.00	0.00	0.000000	0.000000	0.00	0.00	0.024691	0.024691	0.000000	0.000000	0.049383	0.012346	0.000000	0.000000	0.000000	0.000000	0.012346	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.012346	0.012346	0.012346	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.012346	0.00	0.000000	0.00	0.00	0.000000	0.012346	0.000000	0.000000	0.000000	0.000000	0.000000	0.012346	0.000000	0.000000	0.012346	0.000000	0.012346	0.000000	0.000000	0.012346	0.000000	0.000000	0.000000	0.012346	0.000000	0.024691	0.000000	0.024691	0.00	0.000000	0.012346	0.000000	0.000000	0.000000	0.00	0.000000	0.012346	0.000000	0.000000	0.000000	0.000000	0.024691	0.000000	0.000000	0.000000	0.000000	0.000000	0.012346	0.000000	0.000000	0.012346	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.012346	0.000000	0.024691	0.000000	0.00	0.000000	0.00000	0.00	0.012346	0.000000	0.000000	0.012346	0.000000	0.00	0.012346
18	M5H	Downtown Toronto	Adelaide, King, Richmond	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.040000	0.000000	0.00	0.00	0.010000	0.010000	0.000000	0.030000	0.000000	0.000000	0.000000	0.000000	0.030000	0.000000	0.030000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.020000	0.000000	0.01	0.020000	0.000000	0.000000	0.01	0.020000	0.010000	0.00	0.000000	0.050000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.030000	0.010000	0.060000	0.000000	0.000000	0.00	0.000000	0.000000	0.020000	0.000000	0.020000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.010000	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.0000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.01	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.00	0.000000	0.000000	0.020000	0.000000	0.000000	0.01	0.00	0.000000	0.01	0.000000	0.010000	0.000000	0.030000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.00	0.00	0.000000	0.030000	0.00	0.00	0.010000	0.010000	0.000000	0.000000	0.000000	0.020000	0.010000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.010000	0.000000	0.000000	0.000000	0.00	0.010000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.01	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.010000	0.000000	0.010000	0.010000	0.01	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.020000	0.000000	0.010000	0.000000	0.000000	0.000000	0.010000	0.000000	0.000000	0.010000	0.000000	0.000000	0.030000	0.000000	0.010000	0.010000	0.000000	0.00	0.000000	0.010000	0.000000	0.000000	0.000000	0.00	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.040000	0.000000	0.000000	0.020000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.040000	0.010000	0.00	0.000000	0.00000	0.00	0.010000	0.000000	0.000000	0.010000	0.000000	0.01	0.000000
19	M5J	Downtown Toronto	Harbourfront East, Toronto Islands, Union Station	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.05	0.00	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.030000	0.010000	0.020000	0.01	0.010000	0.000000	0.010000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.00	0.000000	0.030000	0.010000	0.00	0.000000	0.000000	0.00	0.000000	0.040000	0.000000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.140000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.010000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.0000	0.000000	0.000000	0.010000	0.000000	0.000000	0.010000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.020000	0.00000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.020000	0.000000	0.0	0.00	0.00	0.000000	0.050000	0.01	0.00	0.010000	0.010000	0.010000	0.000000	0.030000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.01	0.000000	0.000000	0.00	0.000000	0.010000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.01	0.000000	0.000000	0.000000	0.000000	0.020000	0.010000	0.010000	0.000000	0.000000	0.010000	0.00	0.000000	0.00	0.00	0.000000	0.020000	0.010000	0.000000	0.000000	0.000000	0.000000	0.040000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.030000	0.000000	0.010000	0.000000	0.010000	0.03	0.000000	0.010000	0.000000	0.000000	0.000000	0.01	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.020000	0.020000	0.000000	0.000000	0.010000	0.000000	0.010000	0.010000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.010000	0.000000	0.000000	0.010000	0.00	0.000000	0.00000	0.01	0.010000	0.000000	0.000000	0.010000	0.000000	0.00	0.000000
20	M5K	Downtown Toronto	Design Exchange, Toronto Dominion Centre	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.040000	0.000000	0.00	0.00	0.010000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.020000	0.00	0.010000	0.000000	0.020000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.00	0.010000	0.000000	0.000000	0.00	0.020000	0.000000	0.00	0.000000	0.080000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.140000	0.000000	0.000000	0.00	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.030000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.01	0.000000	0.000000	0.010000	0.010000	0.00000	0.000000	0.00	0.000000	0.000000	0.030000	0.000000	0.000000	0.01	0.00	0.000000	0.01	0.000000	0.010000	0.000000	0.030000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.00	0.00	0.000000	0.080000	0.01	0.00	0.010000	0.000000	0.000000	0.000000	0.030000	0.020000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.010000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.010000	0.000000	0.010000	0.010000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.040000	0.000000	0.010000	0.000000	0.010000	0.00	0.000000	0.020000	0.000000	0.010000	0.000000	0.00	0.000000	0.000000	0.000000	0.010000	0.000000	0.000000	0.010000	0.010000	0.010000	0.020000	0.000000	0.000000	0.020000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.010000	0.000000	0.010000	0.010000	0.00	0.000000	0.00000	0.01	0.000000	0.000000	0.000000	0.010000	0.000000	0.00	0.000000
21	M5L	Downtown Toronto	Commerce Court, Victoria Hotel	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.040000	0.000000	0.00	0.00	0.010000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.030000	0.000000	0.020000	0.00	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.00	0.010000	0.000000	0.000000	0.01	0.010000	0.000000	0.00	0.000000	0.070000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.020000	0.100000	0.000000	0.000000	0.00	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.030000	0.010000	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.01	0.010000	0.010000	0.010000	0.010000	0.00000	0.000000	0.00	0.000000	0.000000	0.030000	0.000000	0.000000	0.01	0.00	0.000000	0.01	0.000000	0.010000	0.000000	0.030000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.00	0.00	0.000000	0.060000	0.01	0.00	0.010000	0.000000	0.000000	0.010000	0.020000	0.020000	0.010000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.01	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.060000	0.000000	0.020000	0.000000	0.010000	0.00	0.000000	0.030000	0.000000	0.010000	0.000000	0.00	0.000000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.030000	0.000000	0.000000	0.000000	0.00	0.00	0.010000	0.000000	0.00	0.000000	0.010000	0.000000	0.020000	0.000000	0.00	0.000000	0.00000	0.00	0.000000	0.000000	0.000000	0.010000	0.000000	0.00	0.000000
22	M5N	Central Toronto	Roselawn	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.00	0.500000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.5	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.00000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000
23	M5P	Central Toronto	Forest Hill North, Forest Hill West	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.250000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.250000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.250000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.25000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000
24	M5R	Central Toronto	The Annex, North Midtown, Yorkville	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.043478	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.043478	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.043478	0.000000	0.00	0.000000	0.130435	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.130435	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.043478	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.043478	0.000000	0.0	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.043478	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.043478	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.043478	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.043478	0.000000	0.000000	0.000000	0.043478	0.000000	0.086957	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.043478	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.130435	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.00000	0.00	0.043478	0.000000	0.000000	0.000000	0.000000	0.00	0.000000
25	M5S	Downtown Toronto	Harbord, University of Toronto	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.058824	0.000000	0.058824	0.00	0.000000	0.000000	0.029412	0.029412	0.000000	0.000000	0.000000	0.058824	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.117647	0.000000	0.000000	0.000000	0.029412	0.000000	0.000000	0.000000	0.000000	0.000000	0.058824	0.029412	0.029412	0.00	0.029412	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.029412	0.000000	0.00000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.029412	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.029412	0.058824	0.029412	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.029412	0.029412	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.029412	0.029412	0.000000	0.000000	0.000000	0.000000	0.058824	0.000000	0.000000	0.000000	0.029412	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.029412	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.029412	0.00	0.000000	0.00000	0.00	0.000000	0.029412	0.000000	0.000000	0.000000	0.00	0.000000
26	M5T	Downtown Toronto	Chinatown, Grange Park, Kensington Market	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.01	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.010000	0.040000	0.000000	0.070000	0.00	0.000000	0.000000	0.010000	0.000000	0.010000	0.010000	0.000000	0.000000	0.000000	0.00	0.010000	0.010000	0.010000	0.00	0.020000	0.010000	0.00	0.000000	0.070000	0.000000	0.020000	0.010000	0.030000	0.000000	0.000000	0.000000	0.000000	0.010000	0.040000	0.000000	0.000000	0.00	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.020000	0.02	0.000000	0.000000	0.000000	0.01	0.010000	0.04	0.0000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.01	0.010000	0.010000	0.000000	0.000000	0.000000	0.01	0.000000	0.000000	0.000000	0.000000	0.00000	0.010000	0.02	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.010000	0.000000	0.020000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.00	0.01	0.000000	0.000000	0.01	0.01	0.010000	0.000000	0.000000	0.000000	0.010000	0.010000	0.010000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.010000	0.000000	0.000000	0.000000	0.010000	0.01	0.000000	0.000000	0.030000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.00	0.000000	0.01	0.00	0.000000	0.010000	0.000000	0.000000	0.000000	0.010000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.010000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.010000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.01	0.000000	0.000000	0.00	0.000000	0.010000	0.000000	0.010000	0.000000	0.01	0.000000	0.00000	0.00	0.050000	0.000000	0.040000	0.010000	0.000000	0.00	0.000000
27	M5V	Downtown Toronto	CN Tower, Bathurst Quay, Island airport, Harbo...	0.000000	0.000000	0.076923	0.076923	0.076923	0.153846	0.153846	0.153846	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.076923	0.000000	0.076923	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.076923	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.076923	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.00000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000
28	M5W	Downtown Toronto	Stn A PO Boxes 25 The Esplanade	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010526	0.010526	0.00	0.00	0.021053	0.000000	0.000000	0.000000	0.000000	0.010526	0.000000	0.010526	0.021053	0.000000	0.000000	0.00	0.010526	0.000000	0.031579	0.000000	0.010526	0.010526	0.000000	0.010526	0.000000	0.00	0.010526	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.042105	0.000000	0.000000	0.021053	0.000000	0.000000	0.010526	0.000000	0.010526	0.031579	0.094737	0.000000	0.000000	0.00	0.010526	0.000000	0.010526	0.000000	0.021053	0.000000	0.021053	0.000000	0.000000	0.000000	0.010526	0.000000	0.000000	0.00	0.010526	0.000000	0.000000	0.00	0.000000	0.00	0.0000	0.000000	0.000000	0.000000	0.000000	0.021053	0.021053	0.00	0.000000	0.010526	0.000000	0.000000	0.000000	0.00	0.010526	0.010526	0.010526	0.000000	0.00000	0.000000	0.00	0.000000	0.000000	0.010526	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.021053	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.00	0.00	0.010526	0.031579	0.00	0.00	0.000000	0.000000	0.000000	0.010526	0.031579	0.021053	0.010526	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.010526	0.010526	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010526	0.00	0.010526	0.000000	0.010526	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.010526	0.00	0.00	0.000000	0.010526	0.000000	0.000000	0.000000	0.000000	0.000000	0.010526	0.000000	0.000000	0.010526	0.000000	0.000000	0.000000	0.031579	0.000000	0.000000	0.000000	0.000000	0.052632	0.000000	0.010526	0.000000	0.010526	0.00	0.000000	0.031579	0.000000	0.010526	0.000000	0.00	0.000000	0.000000	0.000000	0.010526	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010526	0.000000	0.000000	0.000000	0.00	0.00	0.010526	0.000000	0.00	0.000000	0.010526	0.000000	0.010526	0.000000	0.00	0.000000	0.00000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.010526
29	M5X	Downtown Toronto	First Canadian Place, Underground city	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.040000	0.000000	0.00	0.00	0.010000	0.000000	0.000000	0.030000	0.000000	0.000000	0.000000	0.000000	0.030000	0.000000	0.030000	0.00	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.01	0.000000	0.000000	0.000000	0.01	0.020000	0.010000	0.00	0.000000	0.080000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.080000	0.000000	0.000000	0.00	0.000000	0.000000	0.020000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.030000	0.010000	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.010000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.01	0.000000	0.000000	0.000000	0.010000	0.00000	0.000000	0.00	0.000000	0.000000	0.030000	0.000000	0.000000	0.01	0.00	0.000000	0.01	0.000000	0.010000	0.000000	0.030000	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.00	0.00	0.000000	0.060000	0.01	0.00	0.010000	0.000000	0.000000	0.000000	0.000000	0.020000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.010000	0.000000	0.000000	0.000000	0.00	0.010000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.01	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.010000	0.000000	0.010000	0.000000	0.01	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.020000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.050000	0.000000	0.010000	0.000000	0.010000	0.00	0.000000	0.030000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.010000	0.000000	0.000000	0.000000	0.000000	0.030000	0.000000	0.000000	0.010000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.010000	0.000000	0.010000	0.010000	0.00	0.000000	0.00000	0.01	0.010000	0.000000	0.000000	0.010000	0.000000	0.00	0.000000
30	M6G	Downtown Toronto	Christie	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.066667	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.200000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.066667	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.066667	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.066667	0.000000	0.000000	0.00	0.000000	0.00	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.200000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.066667	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.066667	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.133333	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.066667	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.00000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000
31	M6H	West Toronto	Dovercourt Village, Dufferin	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.105263	0.052632	0.052632	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.052632	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.052632	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.052632	0.000000	0.00	0.000000	0.00	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.052632	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.052632	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.052632	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.052632	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.052632	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.052632	0.000000	0.000000	0.000000	0.105263	0.000000	0.000000	0.000000	0.000000	0.000000	0.052632	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.052632	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.105263	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.00000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000
32	M6J	West Toronto	Little Portugal, Trinity	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.015385	0.000000	0.00	0.00	0.015385	0.000000	0.000000	0.046154	0.000000	0.000000	0.000000	0.000000	0.030769	0.000000	0.123077	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.015385	0.000000	0.000000	0.015385	0.00	0.000000	0.015385	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.046154	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.030769	0.046154	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.015385	0.015385	0.000000	0.000000	0.000000	0.000000	0.00	0.015385	0.000000	0.015385	0.00	0.000000	0.00	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.015385	0.000000	0.00000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.015385	0.00	0.000000	0.015385	0.000000	0.000000	0.015385	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.00	0.00	0.000000	0.000000	0.00	0.00	0.015385	0.000000	0.000000	0.000000	0.015385	0.000000	0.000000	0.000000	0.000000	0.015385	0.015385	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.015385	0.015385	0.000000	0.00	0.000000	0.061538	0.000000	0.000000	0.015385	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.015385	0.000000	0.015385	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.015385	0.000000	0.000000	0.000000	0.000000	0.000000	0.030769	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.015385	0.000000	0.015385	0.000000	0.000000	0.046154	0.000000	0.000000	0.015385	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.015385	0.000000	0.000000	0.000000	0.015385	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.015385	0.000000	0.000000	0.000000	0.015385	0.00	0.000000	0.00000	0.00	0.015385	0.000000	0.030769	0.015385	0.000000	0.00	0.015385
33	M6K	West Toronto	Brockton, Exhibition Place, Parkdale Village	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.052632	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.105263	0.000000	0.000000	0.00	0.000000	0.052632	0.00	0.000000	0.105263	0.000000	0.052632	0.000000	0.000000	0.000000	0.000000	0.052632	0.000000	0.000000	0.105263	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.052632	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00000	0.052632	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.052632	0.052632	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.052632	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.052632	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.052632	0.000000	0.052632	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.052632	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.00000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000
34	M6P	West Toronto	High Park, The Junction South	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.043478	0.00	0.00	0.000000	0.000000	0.043478	0.000000	0.000000	0.000000	0.000000	0.000000	0.043478	0.000000	0.043478	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.043478	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.086957	0.043478	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.043478	0.000000	0.000000	0.00	0.000000	0.00	0.0000	0.000000	0.000000	0.000000	0.000000	0.000000	0.043478	0.00	0.000000	0.000000	0.043478	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.043478	0.00000	0.043478	0.00	0.000000	0.000000	0.043478	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.043478	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.043478	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.086957	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.043478	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.043478	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.043478	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.043478	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.043478	0.000000	0.00	0.000000	0.00000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000
35	M6R	West Toronto	Parkdale, Roncesvalles	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.062500	0.062500	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.062500	0.000000	0.00	0.125000	0.000000	0.000000	0.00	0.062500	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.062500	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.062500	0.000000	0.000000	0.000000	0.000000	0.062500	0.00	0.000000	0.000000	0.062500	0.00	0.000000	0.00	0.0625	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.125000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.062500	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.062500	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.062500	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.00000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000
36	M6S	West Toronto	Runnymede, Swansea	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.025641	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.025641	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.025641	0.00	0.000000	0.076923	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.102564	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.025641	0.00	0.025641	0.000000	0.000000	0.00	0.000000	0.00	0.0000	0.000000	0.000000	0.000000	0.025641	0.000000	0.000000	0.00	0.025641	0.000000	0.000000	0.025641	0.025641	0.00	0.000000	0.000000	0.025641	0.000000	0.00000	0.000000	0.00	0.000000	0.000000	0.025641	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.025641	0.000000	0.025641	0.025641	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.025641	0.000000	0.051282	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.025641	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.025641	0.000000	0.076923	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.025641	0.000000	0.000000	0.000000	0.000000	0.025641	0.000000	0.000000	0.000000	0.025641	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.025641	0.000000	0.000000	0.025641	0.000000	0.025641	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.051282	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.025641	0.000000	0.000000	0.000000	0.00	0.000000	0.00000	0.00	0.025641	0.000000	0.000000	0.000000	0.000000	0.00	0.000000
37	M7Y	East Toronto	Business Reply Mail Processing Centre 969 Eastern	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.052632	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.052632	0.000000	0.00	0.000000	0.052632	0.00	0.052632	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.052632	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.0000	0.000000	0.000000	0.000000	0.000000	0.052632	0.052632	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.00000	0.000000	0.00	0.052632	0.052632	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.0	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.052632	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.052632	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.052632	0.000000	0.000000	0.000000	0.000000	0.000000	0.052632	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.052632	0.000000	0.052632	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.052632	0.00	0.052632	0.000000	0.000000	0.000000	0.000000	0.000000	0.052632	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.00000	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.052632

Now let's create the new dataframe and display the top 10 venues for each PostalCode.

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(38, 13)

In [ ]:
PostalCode	Borough	Neighborhoods	1st Most Common Venue	2nd Most Common Venue	3rd Most Common Venue	4th Most Common Venue	5th Most Common Venue	6th Most Common Venue	7th Most Common Venue	8th Most Common Venue	9th Most Common Venue	10th Most Common Venue
0	M4E	East Toronto	The Beaches	Neighborhood	Coffee Shop	Pub	Other Great Outdoors	Yoga Studio	Filipino Restaurant	Fast Food Restaurant	Farmers Market	Falafel Restaurant	Event Space
1	M4K	East Toronto	The Danforth West, Riverdale	Greek Restaurant	Coffee Shop	Ice Cream Shop	Bookstore	Italian Restaurant	Pizza Place	Diner	Dessert Shop	Pub	Restaurant
2	M4L	East Toronto	The Beaches West, India Bazaar	Park	Sandwich Place	Gym	Light Rail Station	Sushi Restaurant	Movie Theater	Brewery	Italian Restaurant	Food & Drink Shop	Pub
3	M4M	East Toronto	Studio District	Café	Coffee Shop	Bakery	Italian Restaurant	American Restaurant	Clothing Store	Bookstore	Brewery	Seafood Restaurant	Sandwich Place
4	M4N	Central Toronto	Lawrence Park	Park	Swim School	Dim Sum Restaurant	Bus Line	Yoga Studio	Doner Restaurant	Filipino Restaurant	Fast Food Restaurant	Farmers Market	Falafel Restaurant
5	M4P	Central Toronto	Davisville North	Hotel	Sandwich Place	Burger Joint	Breakfast Spot	Park	Food & Drink Shop	Gym	Clothing Store	Dumpling Restaurant	Eastern European Restaurant
6	M4R	Central Toronto	North Toronto West	Sporting Goods Shop	Coffee Shop	Yoga Studio	Health & Beauty Service	Clothing Store	Chinese Restaurant	Dessert Shop	Diner	Rental Car Location	Salon / Barbershop
7	M4S	Central Toronto	Davisville	Dessert Shop	Sandwich Place	Italian Restaurant	Pharmacy	Sushi Restaurant	Coffee Shop	Café	Pizza Place	Seafood Restaurant	Farmers Market
8	M4T	Central Toronto	Moore Park, Summerhill East	Playground	Park	Tennis Court	Discount Store	Filipino Restaurant	Fast Food Restaurant	Farmers Market	Falafel Restaurant	Event Space	Ethiopian Restaurant
9	M4V	Central Toronto	Deer Park, Forest Hill SE, Rathnelly, South Hi...	Coffee Shop	Pub	Pizza Place	American Restaurant	Light Rail Station	Sports Bar	Fried Chicken Joint	Bagel Shop	Supermarket	Sushi Restaurant
10	M4W	Downtown Toronto	Rosedale	Park	Playground	Trail	Discount Store	Filipino Restaurant	Fast Food Restaurant	Farmers Market	Falafel Restaurant	Event Space	Ethiopian Restaurant
11	M4X	Downtown Toronto	Cabbagetown, St. James Town	Restaurant	Coffee Shop	Pub	Pizza Place	Café	Bakery	Italian Restaurant	Chinese Restaurant	Jewelry Store	Bank
12	M4Y	Downtown Toronto	Church and Wellesley	Japanese Restaurant	Sushi Restaurant	Coffee Shop	Gay Bar	Restaurant	Burger Joint	Café	Pub	Men's Store	Mediterranean Restaurant
13	M5A	Downtown Toronto	Harbourfront, Regent Park	Coffee Shop	Bakery	Pub	Park	Café	Theater	Mexican Restaurant	Breakfast Spot	French Restaurant	Chocolate Shop
14	M5B	Downtown Toronto	Ryerson, Garden District	Coffee Shop	Clothing Store	Café	Cosmetics Shop	Middle Eastern Restaurant	Thai Restaurant	Japanese Restaurant	Diner	Pizza Place	Plaza
15	M5C	Downtown Toronto	St. James Town	Coffee Shop	Restaurant	Café	Hotel	Clothing Store	Bakery	Italian Restaurant	Cosmetics Shop	Park	Cocktail Bar
16	M5E	Downtown Toronto	Berczy Park	Coffee Shop	Restaurant	Cocktail Bar	Bakery	Seafood Restaurant	Italian Restaurant	Beer Bar	Steakhouse	Café	Cheese Shop
17	M5G	Downtown Toronto	Central Bay Street	Coffee Shop	Café	Italian Restaurant	Burger Joint	Bar	Falafel Restaurant	Thai Restaurant	Bubble Tea Shop	Salad Place	Chinese Restaurant
18	M5H	Downtown Toronto	Adelaide, King, Richmond	Coffee Shop	Café	Steakhouse	Thai Restaurant	American Restaurant	Clothing Store	Bakery	Gym	Restaurant	Asian Restaurant
19	M5J	Downtown Toronto	Harbourfront East, Toronto Islands, Union Station	Coffee Shop	Hotel	Aquarium	Café	Pizza Place	Restaurant	Italian Restaurant	Bakery	Scenic Lookout	Brewery
20	M5K	Downtown Toronto	Design Exchange, Toronto Dominion Centre	Coffee Shop	Hotel	Café	Restaurant	American Restaurant	Gastropub	Gym	Italian Restaurant	Deli / Bodega	Bar
21	M5L	Downtown Toronto	Commerce Court, Victoria Hotel	Coffee Shop	Café	Hotel	Restaurant	American Restaurant	Gym	Steakhouse	Bakery	Seafood Restaurant	Gastropub
22	M5N	Central Toronto	Roselawn	Home Service	Garden	Doner Restaurant	Fish & Chips Shop	Filipino Restaurant	Fast Food Restaurant	Farmers Market	Falafel Restaurant	Event Space	Ethiopian Restaurant
23	M5P	Central Toronto	Forest Hill North, Forest Hill West	Jewelry Store	Park	Trail	Sushi Restaurant	Yoga Studio	Filipino Restaurant	Fast Food Restaurant	Farmers Market	Falafel Restaurant	Event Space
24	M5R	Central Toronto	The Annex, North Midtown, Yorkville	Sandwich Place	Coffee Shop	Café	Pizza Place	Cosmetics Shop	American Restaurant	Indian Restaurant	BBQ Joint	Liquor Store	Jewish Restaurant
25	M5S	Downtown Toronto	Harbord, University of Toronto	Café	Coffee Shop	Bakery	Bookstore	Restaurant	Bar	Japanese Restaurant	Nightclub	Noodle House	Comfort Food Restaurant
26	M5T	Downtown Toronto	Chinatown, Grange Park, Kensington Market	Bar	Café	Vegetarian / Vegan Restaurant	Coffee Shop	Vietnamese Restaurant	Bakery	Dumpling Restaurant	Mexican Restaurant	Chinese Restaurant	Burger Joint
27	M5V	Downtown Toronto	CN Tower, Bathurst Quay, Island airport, Harbo...	Airport Lounge	Airport Service	Airport Terminal	Boat or Ferry	Boutique	Airport	Airport Food Court	Airport Gate	Harbor / Marina	Sculpture Garden
28	M5W	Downtown Toronto	Stn A PO Boxes 25 The Esplanade	Coffee Shop	Restaurant	Café	Cocktail Bar	Seafood Restaurant	Pub	Hotel	Beer Bar	Italian Restaurant	Fast Food Restaurant
29	M5X	Downtown Toronto	First Canadian Place, Underground city	Café	Coffee Shop	Hotel	Restaurant	American Restaurant	Gastropub	Bakery	Steakhouse	Asian Restaurant	Deli / Bodega
30	M6G	Downtown Toronto	Christie	Café	Grocery Store	Park	Convenience Store	Coffee Shop	Baby Store	Nightclub	Diner	Italian Restaurant	Restaurant
31	M6H	West Toronto	Dovercourt Village, Dufferin	Pharmacy	Supermarket	Bakery	Park	Music Venue	Brewery	Liquor Store	Middle Eastern Restaurant	Café	Bar
32	M6J	West Toronto	Little Portugal, Trinity	Bar	Men's Store	Coffee Shop	Café	Asian Restaurant	Restaurant	Cocktail Bar	Pizza Place	Bakery	Vietnamese Restaurant
33	M6K	West Toronto	Brockton, Exhibition Place, Parkdale Village	Café	Coffee Shop	Breakfast Spot	Stadium	Nightclub	Italian Restaurant	Bar	Burrito Place	Caribbean Restaurant	Climbing Gym
34	M6P	West Toronto	High Park, The Junction South	Mexican Restaurant	Café	Fast Food Restaurant	Fried Chicken Joint	Furniture / Home Store	Bakery	Cajun / Creole Restaurant	Speakeasy	Italian Restaurant	Gastropub
35	M6R	West Toronto	Parkdale, Roncesvalles	Breakfast Spot	Gift Shop	Bookstore	Burger Joint	Movie Theater	Bar	Bank	Eastern European Restaurant	Dog Run	Italian Restaurant
36	M6S	West Toronto	Runnymede, Swansea	Coffee Shop	Café	Pizza Place	Italian Restaurant	Sushi Restaurant	Pharmacy	Smoothie Shop	Bookstore	Sandwich Place	Burrito Place
37	M7Y	East Toronto	Business Reply Mail Processing Centre 969 Eastern	Yoga Studio	Auto Workshop	Comic Shop	Pizza Place	Moving Target	Recording Studio	Restaurant	Butcher	Burrito Place	Brewery

<h2>16. Cluster Areas</h2>

Run k-means to cluster the Toronto areas into 5 clusters.

In [ ]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(["PostalCode", "Borough", "Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

In [ ]:
array([0, 0, 0, 0, 3, 0, 0, 0, 2, 0])

In [ ]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
toronto_merged = toronto_df_new.copy()

# add clustering labels
toronto_merged["Cluster Labels"] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"), on="PostalCode")

print(toronto_merged.shape)
toronto_merged.head() # check the last columns!

(38, 16)

In [ ]:
PostalCode	Borough	Neighborhood	Latitude	Longitude	Cluster Labels	1st Most Common Venue	2nd Most Common Venue	3rd Most Common Venue	4th Most Common Venue	5th Most Common Venue	6th Most Common Venue	7th Most Common Venue	8th Most Common Venue	9th Most Common Venue	10th Most Common Venue
0	M4E	East Toronto	The Beaches	43.676357	-79.293031	0	Neighborhood	Coffee Shop	Pub	Other Great Outdoors	Yoga Studio	Filipino Restaurant	Fast Food Restaurant	Farmers Market	Falafel Restaurant	Event Space
1	M4K	East Toronto	The Danforth West, Riverdale	43.679557	-79.352188	0	Greek Restaurant	Coffee Shop	Ice Cream Shop	Bookstore	Italian Restaurant	Pizza Place	Diner	Dessert Shop	Pub	Restaurant
2	M4L	East Toronto	The Beaches West, India Bazaar	43.668999	-79.315572	0	Park	Sandwich Place	Gym	Light Rail Station	Sushi Restaurant	Movie Theater	Brewery	Italian Restaurant	Food & Drink Shop	Pub
3	M4M	East Toronto	Studio District	43.659526	-79.340923	0	Café	Coffee Shop	Bakery	Italian Restaurant	American Restaurant	Clothing Store	Bookstore	Brewery	Seafood Restaurant	Sandwich Place
4	M4N	Central Toronto	Lawrence Park	43.728020	-79.388790	3	Park	Swim School	Dim Sum Restaurant	Bus Line	Yoga Studio	Doner Restaurant	Filipino Restaurant	Fast Food Restaurant	Farmers Market	Falafel Restaurant

In [ ]:
# sort the results by Cluster Labels
print(toronto_merged.shape)
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged

(38, 16)

In [ ]:
PostalCode	Borough	Neighborhood	Latitude	Longitude	Cluster Labels	1st Most Common Venue	2nd Most Common Venue	3rd Most Common Venue	4th Most Common Venue	5th Most Common Venue	6th Most Common Venue	7th Most Common Venue	8th Most Common Venue	9th Most Common Venue	10th Most Common Venue
0	M4E	East Toronto	The Beaches	43.676357	-79.293031	0	Neighborhood	Coffee Shop	Pub	Other Great Outdoors	Yoga Studio	Filipino Restaurant	Fast Food Restaurant	Farmers Market	Falafel Restaurant	Event Space
20	M5K	Downtown Toronto	Design Exchange, Toronto Dominion Centre	43.647177	-79.381576	0	Coffee Shop	Hotel	Café	Restaurant	American Restaurant	Gastropub	Gym	Italian Restaurant	Deli / Bodega	Bar
21	M5L	Downtown Toronto	Commerce Court, Victoria Hotel	43.648198	-79.379817	0	Coffee Shop	Café	Hotel	Restaurant	American Restaurant	Gym	Steakhouse	Bakery	Seafood Restaurant	Gastropub
24	M5R	Central Toronto	The Annex, North Midtown, Yorkville	43.672710	-79.405678	0	Sandwich Place	Coffee Shop	Café	Pizza Place	Cosmetics Shop	American Restaurant	Indian Restaurant	BBQ Joint	Liquor Store	Jewish Restaurant
25	M5S	Downtown Toronto	Harbord, University of Toronto	43.662696	-79.400049	0	Café	Coffee Shop	Bakery	Bookstore	Restaurant	Bar	Japanese Restaurant	Nightclub	Noodle House	Comfort Food Restaurant
26	M5T	Downtown Toronto	Chinatown, Grange Park, Kensington Market	43.653206	-79.400049	0	Bar	Café	Vegetarian / Vegan Restaurant	Coffee Shop	Vietnamese Restaurant	Bakery	Dumpling Restaurant	Mexican Restaurant	Chinese Restaurant	Burger Joint
27	M5V	Downtown Toronto	CN Tower, Bathurst Quay, Island airport, Harbo...	43.628947	-79.394420	0	Airport Lounge	Airport Service	Airport Terminal	Boat or Ferry	Boutique	Airport	Airport Food Court	Airport Gate	Harbor / Marina	Sculpture Garden
19	M5J	Downtown Toronto	Harbourfront East, Toronto Islands, Union Station	43.640816	-79.381752	0	Coffee Shop	Hotel	Aquarium	Café	Pizza Place	Restaurant	Italian Restaurant	Bakery	Scenic Lookout	Brewery
28	M5W	Downtown Toronto	Stn A PO Boxes 25 The Esplanade	43.646435	-79.374846	0	Coffee Shop	Restaurant	Café	Cocktail Bar	Seafood Restaurant	Pub	Hotel	Beer Bar	Italian Restaurant	Fast Food Restaurant
30	M6G	Downtown Toronto	Christie	43.669542	-79.422564	0	Café	Grocery Store	Park	Convenience Store	Coffee Shop	Baby Store	Nightclub	Diner	Italian Restaurant	Restaurant
31	M6H	West Toronto	Dovercourt Village, Dufferin	43.669005	-79.442259	0	Pharmacy	Supermarket	Bakery	Park	Music Venue	Brewery	Liquor Store	Middle Eastern Restaurant	Café	Bar
32	M6J	West Toronto	Little Portugal, Trinity	43.647927	-79.419750	0	Bar	Men's Store	Coffee Shop	Café	Asian Restaurant	Restaurant	Cocktail Bar	Pizza Place	Bakery	Vietnamese Restaurant
33	M6K	West Toronto	Brockton, Exhibition Place, Parkdale Village	43.636847	-79.428191	0	Café	Coffee Shop	Breakfast Spot	Stadium	Nightclub	Italian Restaurant	Bar	Burrito Place	Caribbean Restaurant	Climbing Gym
34	M6P	West Toronto	High Park, The Junction South	43.661608	-79.464763	0	Mexican Restaurant	Café	Fast Food Restaurant	Fried Chicken Joint	Furniture / Home Store	Bakery	Cajun / Creole Restaurant	Speakeasy	Italian Restaurant	Gastropub
35	M6R	West Toronto	Parkdale, Roncesvalles	43.648960	-79.456325	0	Breakfast Spot	Gift Shop	Bookstore	Burger Joint	Movie Theater	Bar	Bank	Eastern European Restaurant	Dog Run	Italian Restaurant
29	M5X	Downtown Toronto	First Canadian Place, Underground city	43.648429	-79.382280	0	Café	Coffee Shop	Hotel	Restaurant	American Restaurant	Gastropub	Bakery	Steakhouse	Asian Restaurant	Deli / Bodega
36	M6S	West Toronto	Runnymede, Swansea	43.651571	-79.484450	0	Coffee Shop	Café	Pizza Place	Italian Restaurant	Sushi Restaurant	Pharmacy	Smoothie Shop	Bookstore	Sandwich Place	Burrito Place
18	M5H	Downtown Toronto	Adelaide, King, Richmond	43.650571	-79.384568	0	Coffee Shop	Café	Steakhouse	Thai Restaurant	American Restaurant	Clothing Store	Bakery	Gym	Restaurant	Asian Restaurant
16	M5E	Downtown Toronto	Berczy Park	43.644771	-79.373306	0	Coffee Shop	Restaurant	Cocktail Bar	Bakery	Seafood Restaurant	Italian Restaurant	Beer Bar	Steakhouse	Café	Cheese Shop
1	M4K	East Toronto	The Danforth West, Riverdale	43.679557	-79.352188	0	Greek Restaurant	Coffee Shop	Ice Cream Shop	Bookstore	Italian Restaurant	Pizza Place	Diner	Dessert Shop	Pub	Restaurant
2	M4L	East Toronto	The Beaches West, India Bazaar	43.668999	-79.315572	0	Park	Sandwich Place	Gym	Light Rail Station	Sushi Restaurant	Movie Theater	Brewery	Italian Restaurant	Food & Drink Shop	Pub
3	M4M	East Toronto	Studio District	43.659526	-79.340923	0	Café	Coffee Shop	Bakery	Italian Restaurant	American Restaurant	Clothing Store	Bookstore	Brewery	Seafood Restaurant	Sandwich Place
5	M4P	Central Toronto	Davisville North	43.712751	-79.390197	0	Hotel	Sandwich Place	Burger Joint	Breakfast Spot	Park	Food & Drink Shop	Gym	Clothing Store	Dumpling Restaurant	Eastern European Restaurant
6	M4R	Central Toronto	North Toronto West	43.715383	-79.405678	0	Sporting Goods Shop	Coffee Shop	Yoga Studio	Health & Beauty Service	Clothing Store	Chinese Restaurant	Dessert Shop	Diner	Rental Car Location	Salon / Barbershop
7	M4S	Central Toronto	Davisville	43.704324	-79.388790	0	Dessert Shop	Sandwich Place	Italian Restaurant	Pharmacy	Sushi Restaurant	Coffee Shop	Café	Pizza Place	Seafood Restaurant	Farmers Market
17	M5G	Downtown Toronto	Central Bay Street	43.657952	-79.387383	0	Coffee Shop	Café	Italian Restaurant	Burger Joint	Bar	Falafel Restaurant	Thai Restaurant	Bubble Tea Shop	Salad Place	Chinese Restaurant
9	M4V	Central Toronto	Deer Park, Forest Hill SE, Rathnelly, South Hi...	43.686412	-79.400049	0	Coffee Shop	Pub	Pizza Place	American Restaurant	Light Rail Station	Sports Bar	Fried Chicken Joint	Bagel Shop	Supermarket	Sushi Restaurant
11	M4X	Downtown Toronto	Cabbagetown, St. James Town	43.667967	-79.367675	0	Restaurant	Coffee Shop	Pub	Pizza Place	Café	Bakery	Italian Restaurant	Chinese Restaurant	Jewelry Store	Bank
12	M4Y	Downtown Toronto	Church and Wellesley	43.665860	-79.383160	0	Japanese Restaurant	Sushi Restaurant	Coffee Shop	Gay Bar	Restaurant	Burger Joint	Café	Pub	Men's Store	Mediterranean Restaurant
13	M5A	Downtown Toronto	Harbourfront, Regent Park	43.654260	-79.360636	0	Coffee Shop	Bakery	Pub	Park	Café	Theater	Mexican Restaurant	Breakfast Spot	French Restaurant	Chocolate Shop
14	M5B	Downtown Toronto	Ryerson, Garden District	43.657162	-79.378937	0	Coffee Shop	Clothing Store	Café	Cosmetics Shop	Middle Eastern Restaurant	Thai Restaurant	Japanese Restaurant	Diner	Pizza Place	Plaza
15	M5C	Downtown Toronto	St. James Town	43.651494	-79.375418	0	Coffee Shop	Restaurant	Café	Hotel	Clothing Store	Bakery	Italian Restaurant	Cosmetics Shop	Park	Cocktail Bar
37	M7Y	East Toronto	Business Reply Mail Processing Centre 969 Eastern	43.662744	-79.321558	0	Yoga Studio	Auto Workshop	Comic Shop	Pizza Place	Moving Target	Recording Studio	Restaurant	Butcher	Burrito Place	Brewery
22	M5N	Central Toronto	Roselawn	43.711695	-79.416936	1	Home Service	Garden	Doner Restaurant	Fish & Chips Shop	Filipino Restaurant	Fast Food Restaurant	Farmers Market	Falafel Restaurant	Event Space	Ethiopian Restaurant
8	M4T	Central Toronto	Moore Park, Summerhill East	43.689574	-79.383160	2	Playground	Park	Tennis Court	Discount Store	Filipino Restaurant	Fast Food Restaurant	Farmers Market	Falafel Restaurant	Event Space	Ethiopian Restaurant
10	M4W	Downtown Toronto	Rosedale	43.679563	-79.377529	2	Park	Playground	Trail	Discount Store	Filipino Restaurant	Fast Food Restaurant	Farmers Market	Falafel Restaurant	Event Space	Ethiopian Restaurant
4	M4N	Central Toronto	Lawrence Park	43.728020	-79.388790	3	Park	Swim School	Dim Sum Restaurant	Bus Line	Yoga Studio	Doner Restaurant	Filipino Restaurant	Fast Food Restaurant	Farmers Market	Falafel Restaurant
23	M5P	Central Toronto	Forest Hill North, Forest Hill West	43.696948	-79.411307	4	Jewelry Store	Park	Trail	Sushi Restaurant	Yoga Studio	Filipino Restaurant	Fast Food Restaurant	Farmers Market	Falafel Restaurant	Event Space

<h2>Finally, let's visualize the resulting clusters</h2>

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Borough'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h2>17. Examine Clusters</h2>

<h2>Cluster 1</h2>

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
Borough	Cluster Labels	1st Most Common Venue	2nd Most Common Venue	3rd Most Common Venue	4th Most Common Venue	5th Most Common Venue	6th Most Common Venue	7th Most Common Venue	8th Most Common Venue	9th Most Common Venue	10th Most Common Venue
0	East Toronto	0	Neighborhood	Coffee Shop	Pub	Other Great Outdoors	Yoga Studio	Filipino Restaurant	Fast Food Restaurant	Farmers Market	Falafel Restaurant	Event Space
20	Downtown Toronto	0	Coffee Shop	Hotel	Café	Restaurant	American Restaurant	Gastropub	Gym	Italian Restaurant	Deli / Bodega	Bar
21	Downtown Toronto	0	Coffee Shop	Café	Hotel	Restaurant	American Restaurant	Gym	Steakhouse	Bakery	Seafood Restaurant	Gastropub
24	Central Toronto	0	Sandwich Place	Coffee Shop	Café	Pizza Place	Cosmetics Shop	American Restaurant	Indian Restaurant	BBQ Joint	Liquor Store	Jewish Restaurant
25	Downtown Toronto	0	Café	Coffee Shop	Bakery	Bookstore	Restaurant	Bar	Japanese Restaurant	Nightclub	Noodle House	Comfort Food Restaurant
26	Downtown Toronto	0	Bar	Café	Vegetarian / Vegan Restaurant	Coffee Shop	Vietnamese Restaurant	Bakery	Dumpling Restaurant	Mexican Restaurant	Chinese Restaurant	Burger Joint
27	Downtown Toronto	0	Airport Lounge	Airport Service	Airport Terminal	Boat or Ferry	Boutique	Airport	Airport Food Court	Airport Gate	Harbor / Marina	Sculpture Garden
19	Downtown Toronto	0	Coffee Shop	Hotel	Aquarium	Café	Pizza Place	Restaurant	Italian Restaurant	Bakery	Scenic Lookout	Brewery
28	Downtown Toronto	0	Coffee Shop	Restaurant	Café	Cocktail Bar	Seafood Restaurant	Pub	Hotel	Beer Bar	Italian Restaurant	Fast Food Restaurant
30	Downtown Toronto	0	Café	Grocery Store	Park	Convenience Store	Coffee Shop	Baby Store	Nightclub	Diner	Italian Restaurant	Restaurant
31	West Toronto	0	Pharmacy	Supermarket	Bakery	Park	Music Venue	Brewery	Liquor Store	Middle Eastern Restaurant	Café	Bar
32	West Toronto	0	Bar	Men's Store	Coffee Shop	Café	Asian Restaurant	Restaurant	Cocktail Bar	Pizza Place	Bakery	Vietnamese Restaurant
33	West Toronto	0	Café	Coffee Shop	Breakfast Spot	Stadium	Nightclub	Italian Restaurant	Bar	Burrito Place	Caribbean Restaurant	Climbing Gym
34	West Toronto	0	Mexican Restaurant	Café	Fast Food Restaurant	Fried Chicken Joint	Furniture / Home Store	Bakery	Cajun / Creole Restaurant	Speakeasy	Italian Restaurant	Gastropub
35	West Toronto	0	Breakfast Spot	Gift Shop	Bookstore	Burger Joint	Movie Theater	Bar	Bank	Eastern European Restaurant	Dog Run	Italian Restaurant
29	Downtown Toronto	0	Café	Coffee Shop	Hotel	Restaurant	American Restaurant	Gastropub	Bakery	Steakhouse	Asian Restaurant	Deli / Bodega
36	West Toronto	0	Coffee Shop	Café	Pizza Place	Italian Restaurant	Sushi Restaurant	Pharmacy	Smoothie Shop	Bookstore	Sandwich Place	Burrito Place
18	Downtown Toronto	0	Coffee Shop	Café	Steakhouse	Thai Restaurant	American Restaurant	Clothing Store	Bakery	Gym	Restaurant	Asian Restaurant
16	Downtown Toronto	0	Coffee Shop	Restaurant	Cocktail Bar	Bakery	Seafood Restaurant	Italian Restaurant	Beer Bar	Steakhouse	Café	Cheese Shop
1	East Toronto	0	Greek Restaurant	Coffee Shop	Ice Cream Shop	Bookstore	Italian Restaurant	Pizza Place	Diner	Dessert Shop	Pub	Restaurant
2	East Toronto	0	Park	Sandwich Place	Gym	Light Rail Station	Sushi Restaurant	Movie Theater	Brewery	Italian Restaurant	Food & Drink Shop	Pub
3	East Toronto	0	Café	Coffee Shop	Bakery	Italian Restaurant	American Restaurant	Clothing Store	Bookstore	Brewery	Seafood Restaurant	Sandwich Place
5	Central Toronto	0	Hotel	Sandwich Place	Burger Joint	Breakfast Spot	Park	Food & Drink Shop	Gym	Clothing Store	Dumpling Restaurant	Eastern European Restaurant
6	Central Toronto	0	Sporting Goods Shop	Coffee Shop	Yoga Studio	Health & Beauty Service	Clothing Store	Chinese Restaurant	Dessert Shop	Diner	Rental Car Location	Salon / Barbershop
7	Central Toronto	0	Dessert Shop	Sandwich Place	Italian Restaurant	Pharmacy	Sushi Restaurant	Coffee Shop	Café	Pizza Place	Seafood Restaurant	Farmers Market
17	Downtown Toronto	0	Coffee Shop	Café	Italian Restaurant	Burger Joint	Bar	Falafel Restaurant	Thai Restaurant	Bubble Tea Shop	Salad Place	Chinese Restaurant
9	Central Toronto	0	Coffee Shop	Pub	Pizza Place	American Restaurant	Light Rail Station	Sports Bar	Fried Chicken Joint	Bagel Shop	Supermarket	Sushi Restaurant
11	Downtown Toronto	0	Restaurant	Coffee Shop	Pub	Pizza Place	Café	Bakery	Italian Restaurant	Chinese Restaurant	Jewelry Store	Bank
12	Downtown Toronto	0	Japanese Restaurant	Sushi Restaurant	Coffee Shop	Gay Bar	Restaurant	Burger Joint	Café	Pub	Men's Store	Mediterranean Restaurant
13	Downtown Toronto	0	Coffee Shop	Bakery	Pub	Park	Café	Theater	Mexican Restaurant	Breakfast Spot	French Restaurant	Chocolate Shop
14	Downtown Toronto	0	Coffee Shop	Clothing Store	Café	Cosmetics Shop	Middle Eastern Restaurant	Thai Restaurant	Japanese Restaurant	Diner	Pizza Place	Plaza
15	Downtown Toronto	0	Coffee Shop	Restaurant	Café	Hotel	Clothing Store	Bakery	Italian Restaurant	Cosmetics Shop	Park	Cocktail Bar
37	East Toronto	0	Yoga Studio	Auto Workshop	Comic Shop	Pizza Place	Moving Target	Recording Studio	Restaurant	Butcher	Burrito Place	Brewery

<h2>Cluster 2</h2>

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
Borough	Cluster Labels	1st Most Common Venue	2nd Most Common Venue	3rd Most Common Venue	4th Most Common Venue	5th Most Common Venue	6th Most Common Venue	7th Most Common Venue	8th Most Common Venue	9th Most Common Venue	10th Most Common Venue
22	Central Toronto	1	Home Service	Garden	Doner Restaurant	Fish & Chips Shop	Filipino Restaurant	Fast Food Restaurant	Farmers Market	Falafel Restaurant	Event Space	Ethiopian Restaurant

<h2>Cluster 3</h2>

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
Borough	Cluster Labels	1st Most Common Venue	2nd Most Common Venue	3rd Most Common Venue	4th Most Common Venue	5th Most Common Venue	6th Most Common Venue	7th Most Common Venue	8th Most Common Venue	9th Most Common Venue	10th Most Common Venue
8	Central Toronto	2	Playground	Park	Tennis Court	Discount Store	Filipino Restaurant	Fast Food Restaurant	Farmers Market	Falafel Restaurant	Event Space	Ethiopian Restaurant
10	Downtown Toronto	2	Park	Playground	Trail	Discount Store	Filipino Restaurant	Fast Food Restaurant	Farmers Market	Falafel Restaurant	Event Space	Ethiopian Restaurant

<h2>Cluster 4</h2>

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
Borough	Cluster Labels	1st Most Common Venue	2nd Most Common Venue	3rd Most Common Venue	4th Most Common Venue	5th Most Common Venue	6th Most Common Venue	7th Most Common Venue	8th Most Common Venue	9th Most Common Venue	10th Most Common Venue
4	Central Toronto	3	Park	Swim School	Dim Sum Restaurant	Bus Line	Yoga Studio	Doner Restaurant	Filipino Restaurant	Fast Food Restaurant	Farmers Market	Falafel Restaurant

<h2>Cluster 5</h2>

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
Borough	Cluster Labels	1st Most Common Venue	2nd Most Common Venue	3rd Most Common Venue	4th Most Common Venue	5th Most Common Venue	6th Most Common Venue	7th Most Common Venue	8th Most Common Venue	9th Most Common Venue	10th Most Common Venue
23	Central Toronto	4	Jewelry Store	Park	Trail	Sushi Restaurant	Yoga Studio	Filipino Restaurant	Fast Food Restaurant	Farmers Market	Falafel Restaurant	Event Space

<h2>Observations:</h2>

Most of the neighborhoods fall into Cluster 1 which are mostly business areas with cafe, restaurants, supermarkets etc. Cluster 2 is just a garden, Cluster 3 are playground and park, Cluster 4 park and swim school, and lastly Cluster 5 park and trail.